In [ ]:
import multiprocessing
import re
from io import StringIO
import uuid
import os

if not os.path.exists('out'):
    os.makedirs('out')
if not os.path.exists('results'):
    os.makedirs('results')

import pandas as pd
import numpy as np

from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs.latin_design import LatinDesign
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction

In [ ]:
target_name = 'SEIZ_updated'
threads_per_run = multiprocessing.cpu_count()
init_exposures = 100

parameter_space = ParameterSpace([ContinuousParameter('p', 0, 1),
                                  ContinuousParameter('l', 0, 1),
                                  ContinuousParameter('epsilon', 2, 2),
                                  # DiscreteParameter('prop_init_skeptics', np.arange(1, init_exposures-1, 1, dtype=np.int_))
                                  DiscreteParameter('prop_init_skeptics', np.array([50]))
                                ])

In [ ]:
def run_job(p, l, epsilon, init_skeptics, init_adopters):
    with open(f'{target_name}_editable.fred', 'r') as file:
        filedata = file.read()

    vars = {
        'p': p, 
        'l': l, 
        'epsilon': epsilon, 
        'init_skeptic': int(init_skeptics), 
        'init_adopt': int(init_adopters)
    }

    run_uuid = uuid.uuid4()
    var_str = ', '.join(map(lambda item: f'{item[0]}={item[1]}', vars.items()))
    print(run_uuid, var_str)

    with open(f'out/{target_name}_{run_uuid}_out.fred', 'w') as file:
        file.write(filedata.format(**vars))
        
    !fred_job -p out/{target_name}_{run_uuid}_out.fred -k {target_name}_{run_uuid}_run -t {threads_per_run}
    !fred_csv -k {target_name}_{run_uuid}_run > results/{target_name}_{run_uuid}_run.csv

    with open(f'results/{target_name}_{run_uuid}_run.csv', 'r') as file:
        lines = file.readlines()

    with open(f'results/{target_name}_{run_uuid}_run.csv', 'w') as file:
        lines[0] = var_str+'\n'
        file.writelines(lines)

    return pd.read_csv(f'results/{target_name}_{run_uuid}_run.csv', header=2)

In [ ]:
def sample(X, clear_results=False):
    X = np.array(X)
    
    if X.ndim < 2:
        X = np.expand_dims(X, 0)
    elif X.ndim > 2:
        raise Exception(f'X has too many dimensions (ndim={X.ndim}, must be 1 or 2)')
    if X.shape[-1] != len(parameter_space._parameters):
        raise Exception(f'X has the wrong number of variables (variables={X.shape[-1]}, must be {len(parameter_space._parameters)})')

    if clear_results:
        !yes | fred_clear_all_results
    
    Y = []

    for (p, l, epsilon, init_skeptics) in X: # TODO: parallelize this?
        data = run_job(p, l, epsilon, init_skeptics, init_exposures-init_skeptics)
        # Y.append([run['ADOPT.I'].iat[-1] for run in data])
        S_count = data['ADOPT.totS'].iat[0]
        Y.append([(-data['ADOPT.S'].diff()).max() / S_count])
        
    return np.vstack(Y)

In [ ]:
design = LatinDesign(parameter_space)
num_data_points = 10
X = design.get_samples(num_data_points)

In [ ]:
X

In [ ]:
Y = sample(X, clear_results=True)

In [ ]:
Y

In [ ]:
model_gpy = GPRegression(X,Y)
model_emukit = GPyModelWrapper(model_gpy)

In [ ]:
model_variance = ModelVariance(model=model_emukit)
expdesign_loop = ExperimentalDesignLoop(model=model_emukit,
                                        space=parameter_space,
                                        acquisition=model_variance,
                                        batch_size=1)

max_iterations = 50
expdesign_loop.run_loop(sample, max_iterations)

In [ ]:
model_emukit.predict(np.array([[0.5,0.5,2,50]]))